# IPTW(Inverse Probability of Treatment Weightning)
* 逆確率重み付け法
* 傾向スコアの値が0に近いと不安定になる

In [4]:
from sklearn.linear_model import LogisticRegression

import random
import numpy as np
import pandas as pd


from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit


np.random.seed(3655)
#random.seed(3655)

## データの用意

In [22]:
# データ数
num_data = 100000

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数

# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [23]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示



,年齢,性別,CMを見た,購入量
0,36,0,0.0,30.050710
1,55,0,1.0,28.388786
2,47,0,1.0,34.241363
3,47,0,1.0,39.097393
4,58,1,1.0,57.099206


In [24]:
print(df[df["CMを見た"] == 1.0].mean())
print(df[df["CMを見た"] == 0.0].mean())

年齢       52.649075
性別        0.445372
CMを見た     1.000000
購入量      50.736457
dtype: float64
--------
年齢       31.467388
性別        0.597110
CMを見た     0.000000
購入量      66.448382
dtype: float64


## 傾向スコアの出力

In [25]:
# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X,Z)

# 回帰した結果の係数を出力
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.09610993 -0.95874182]]
係数alpha： [-2.89925429]


In [26]:
Z_pre = reg.predict_proba(X)

# 傾向スコアの0になる確率と1になる確率の出力
print(Z_pre[0:5])  
print(Z[0:5])  

[[0.36338474 0.63661526]
 [0.08418543 0.91581457]
 [0.16549295 0.83450705]
 [0.16549295 0.83450705]
 [0.15233802 0.84766198]]
0    0.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: CMを見た, dtype: float64


## 因果のこうk
* 平均処置効果(因果の大きさ)

In [27]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 10.126783722038617
